In [70]:
import numpy as np
from sklearn.metrics import roc_curve
from scipy.optimize import brentq
from scipy.interpolate import interp1d
import pandas as pd
import errno
import cPickle
import os

In [71]:
def create_dir(dir_name):
	try:
		os.makedirs(dir_name)
	except OSError as exception:
		if exception.errno != errno.EEXIST:
			raise

In [72]:
def load_positives_negatives_dict():
    phonemes = [ "Y", "f", "c", "x", "G", "d", "b", "g", "z", "w", "N", "B", "rr", "u", "p", "D", "y", "k", "m", "t", "l", "i", "r", "n", "s", "o", "a", "e"]
    positive_values = [ 53, 682, 405, 590, 222, 773, 528, 887, 189, 743, 911, 428, 491, 1948, 1657, 920, 2453, 1708, 3234, 2938, 3505, 4929, 3650, 7152, 7555, 8040, 10144, 10597]
    negative_values = [10, 10, 105, 153, 643, 89, 395, 114, 997, 500, 443, 1169, 1739, 482, 1055, 2009, 574, 1472, 686, 1542, 1373, 1238, 2617, 476, 480, 2077, 2069, 3484 ]
    total_values = [positive_values[i] + negative_values[i] for i in range(len(positive_values))]
    total_instances = np.float(sum(total_values))
    weighted_values = [v/total_instances for v in total_values]
    positives_dict = dict(zip(phonemes, positive_values))
    negatives_dict = dict(zip(phonemes, negative_values))
    total_dict = dict(zip(phonemes, total_values))
    weighted_dict = dict(zip(phonemes, weighted_values))
    return (total_dict, weighted_dict, positives_dict, negatives_dict)

In [73]:
factors = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

phonemes = [
	"Y", "f", "c", "x", "G", "d", "b", "g", 
	"z", "w", "N", "B", "rr", "u", "p", "D", 
	"y", "k", "m", "t", "l", "i", "r", "n", 
	"s", "o", "a", "e"
]

In [74]:
def create_dir(dir_name):
	try:
		os.makedirs(dir_name)
	except OSError as exception:
		if exception.errno != errno.EEXIST:
			raise

In [75]:
base_dir = "/Users/lmatayoshi/Documents/Projects/tesis_notebooks/kappa_experiments/all_phonemes_experiment/features-combination/"
factors_dict_filename = base_dir + "eers_dict_supervectors_dct"
csvs_dir = base_dir + "csvs/"
create_dir(csvs_dir)
output_filename = csvs_dir + "supervectors_dct_features_combination.csv"

In [76]:
with open(factors_dict_filename, "rb") as f:
    factors_dict = cPickle.load(f)

# Main

In [78]:
def main(factors_dict, csv_filename, factors):
    total_dict, weighted_dict, positives_dict, negatives_dict = load_positives_negatives_dict()
    n_positives = np.array([positives_dict[phoneme] for phoneme in phonemes]).reshape(-1,1)
    n_negatives = np.array([negatives_dict[phoneme] for phoneme in phonemes]).reshape(-1,1)
    n_total = np.array([total_dict[phoneme] for phoneme in phonemes]).reshape(-1,1)
    
    matrix = np.array(phonemes).reshape(-1,1)
    for factor in factors:
        factor_results_dict = factors_dict[factor]
        factor_results = [factor_results_dict[p] for p in phonemes]
        factor_results = np.array(factor_results).reshape(-1,1)
        matrix = np.hstack((matrix, factor_results))
    
    min_indexes = []
    # Find the best proportion by phoneme
    for row in matrix:
        min_index = np.argmin(np.array(row[1:].astype(np.float)))
        min_indexes.append(factors[min_index])
    min_indexes = np.array(min_indexes).reshape(-1,1)
    
    matrix = np.hstack((matrix, min_indexes))
    matrix = np.hstack((matrix, n_positives))
    matrix = np.hstack((matrix, n_negatives))
    matrix = np.hstack((matrix, n_total))
    
    dataframe = pd.DataFrame(matrix[:, 1:], index=phonemes, columns=factors + ['best_proportion','n_positives', 'n_negatives', 'n_total'])
    dataframe = dataframe.astype(np.float)
    dataframe = dataframe.sort_values(by=['n_total'])
    dataframe.round(3).to_csv(path_or_buf=csv_filename, index_label="Phonemes")
    return dataframe

In [79]:
main(factors_dict, output_filename, factors)

,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,best_proportion,n_positives,n_negatives,n_total
Y,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,53.0,10.0,63.0
c,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,405.0,105.0,510.0
f,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,682.0,10.0,692.0
x,0.164407,0.161017,0.170893,0.176471,0.180265,0.183007,0.189542,0.185891,0.191525,0.189542,0.189542,0.1,590.0,153.0,743.0
d,0.363519,0.370370,0.362225,0.404494,0.402329,0.415730,0.426966,0.419993,0.425614,0.419689,0.415730,0.2,773.0,89.0,862.0
G,0.162162,0.174184,0.175676,0.189189,0.190380,0.191291,0.192773,0.206843,0.207207,0.207207,0.211509,0.0,222.0,643.0,865.0
b,0.121519,0.126582,0.141282,0.151899,0.156962,0.162025,0.162025,0.164359,0.167089,0.172152,0.177215,0.0,528.0,395.0,923.0
g,0.234470,0.239008,0.254386,0.245772,0.251409,0.256380,0.263158,0.255994,0.255919,0.260810,0.267884,0.0,887.0,114.0,1001.0
z,0.222222,0.219659,0.227058,0.233299,0.232804,0.235707,0.238095,0.243386,0.245737,0.246740,0.248677,0.1,189.0,997.0,1186.0
w,0.150605,0.150442,0.151210,0.150000,0.156000,0.157738,0.158773,0.158000,0.160000,0.162853,0.163315,0.3,743.0,500.0,1243.0


## previous

In [7]:
def main(eers_dir, csv_filename, values):
    eers_by_value = dict(zip(values, [None] * len(values)))
    
    for v in values:
        with open(eers_dir + str(v), "r") as f:
            lines = f.readlines()
            eer_degree_i = [lines[j].strip().split(":") for j in range(len(lines))]
            eer_degree_i = [value for value in eer_degree_i if len(value) == 2]
            eer_degree_i = dict(eer_degree_i)
            eers_by_value[v] = eer_degree_i
    
    kappa_total_dict, kappa_positives_dict, kappa_negatives_dict = kappa_positives_negatives_dict()
    kappa_total = np.array([kappa_total_dict[phoneme] for phoneme in kappa_phonemes]).reshape(-1,1)
    kappa_positives = np.array([kappa_positives_dict[phoneme] for phoneme in kappa_phonemes]).reshape(-1,1)
    kappa_negatives = np.array([kappa_negatives_dict[phoneme] for phoneme in kappa_phonemes]).reshape(-1,1)
    
    base_matrix = np.array(kappa_phonemes).reshape(-1,1)
    for key in values:
        results_dict = eers_by_value[key]
        results = [float(results_dict[phoneme]) for phoneme in kappa_phonemes]
        results = np.array(results).reshape(-1,1)
        base_matrix = np.hstack((base_matrix, results))
    
    base_matrix = np.hstack((base_matrix, kappa_positives))
    base_matrix = np.hstack((base_matrix, kappa_negatives))
    base_matrix = np.hstack((base_matrix, kappa_total))
    
    dataframe = pd.DataFrame(base_matrix[:, 1:], index=kappa_phonemes, columns=values+['corrects', 'incorrects', 'total'])
    dataframe = dataframe.astype(np.float)
    dataframe = dataframe.sort_values(by=['total'])
    dataframe.round(3).to_csv(path_or_buf=csv_filename, index_label="Phonemes")
    return dataframe


In [9]:
main(
    eers_dir + "legendre_best_degree_eers/",
    csvs_dir + "legendre_best_degree.csv",
    range(7)
)

,0,1,2,3,4,5,6,corrects,incorrects,total
G,0.231726,0.227061,0.220809,0.233281,0.235838,0.233281,0.233281,222.0,643.0,865.0
b,0.200000,0.174242,0.174684,0.189394,0.195076,0.191288,0.183112,528.0,395.0,923.0
w,0.161837,0.158000,0.157102,0.156000,0.160000,0.165899,0.168000,743.0,500.0,1243.0
B,0.225834,0.221963,0.218991,0.224299,0.221557,0.227545,0.235981,428.0,1169.0,1597.0
D,0.250871,0.248382,0.243902,0.240916,0.245652,0.247826,0.252862,920.0,2009.0,2929.0
m,0.190962,0.163265,0.156085,0.155498,0.157434,0.162500,0.164723,3234.0,686.0,3920.0
i,0.260905,0.257050,0.258777,0.254007,0.254211,0.258673,0.259485,4929.0,1238.0,6167.0
s,0.376704,0.328427,0.318750,0.300000,0.295833,0.296360,0.304167,7555.0,480.0,8035.0


In [10]:
main(
    eers_dir + "dct_n_coefficients_eers/",
    csvs_dir + "dct_n_coefficients.csv",
    range(1,11)
)

,1,2,3,4,5,6,7,8,9,10,corrects,incorrects,total
G,0.241058,0.232327,0.234234,0.243243,0.248036,0.238739,0.238739,0.243790,0.250052,0.233766,222.0,643.0,865.0
b,0.201985,0.178030,0.187342,0.186349,0.182278,0.191288,0.187500,0.196669,0.200758,0.202532,528.0,395.0,923.0
w,0.172000,0.170695,0.169583,0.172275,0.170000,0.174000,0.170929,0.170929,0.174056,0.176085,743.0,500.0,1243.0
B,0.238666,0.228826,0.235981,0.238318,0.238519,0.237810,0.235244,0.232094,0.236099,0.231308,428.0,1169.0,1597.0
D,0.260870,0.254351,0.245893,0.252254,0.250455,0.254348,0.260870,0.260826,0.265804,0.265217,920.0,2009.0,2929.0
m,0.181200,0.157081,0.156772,0.163884,0.167638,0.167720,0.167440,0.168522,0.171614,0.171923,3234.0,686.0,3920.0
i,0.277060,0.276527,0.270598,0.269790,0.272875,0.271252,0.270440,0.275444,0.278460,0.279483,4929.0,1238.0,6167.0
s,0.337500,0.314105,0.313538,0.312004,0.312111,0.314063,0.314583,0.320833,0.320833,0.325000,7555.0,480.0,8035.0


In [11]:
main(
    eers_dir + "legendre_best_alpha_eers/",
    csvs_dir + "legendre_best_alpha.csv",
    [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
)

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,corrects,incorrects,total
G,0.222395,0.225505,0.223071,0.220721,0.223950,0.224124,0.223950,0.216216,0.225225,0.222395,222.0,643.0,865.0
b,0.183612,0.175515,0.174363,0.176136,0.179747,0.183712,0.184810,0.187500,0.189873,0.186177,528.0,395.0,923.0
w,0.158348,0.160097,0.161507,0.158000,0.158666,0.159162,0.161420,0.160162,0.161507,0.158816,743.0,500.0,1243.0
B,0.228400,0.226408,0.227371,0.227055,0.221963,0.222412,0.221963,0.221963,0.224123,0.225423,428.0,1169.0,1597.0
D,0.239130,0.234783,0.238043,0.242907,0.240014,0.240014,0.244803,0.244898,0.246889,0.248880,920.0,2009.0,2929.0
m,0.160350,0.161808,0.161581,0.160350,0.161741,0.162028,0.162415,0.166181,0.165170,0.166181,3234.0,686.0,3920.0
i,0.255630,0.255224,0.253222,0.253635,0.256058,0.256058,0.255224,0.256866,0.258006,0.259289,4929.0,1238.0,6167.0
s,0.304167,0.312984,0.320833,0.321305,0.325000,0.330756,0.329167,0.325000,0.324024,0.318750,7555.0,480.0,8035.0


In [12]:
main(
    eers_dir + "features_combination_supervectors_legendre_eers/",
    csvs_dir + "features_combination_supervectors_legendre.csv",
    [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
)

,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,corrects,incorrects,total
G,0.162162,0.166539,0.167963,0.174184,0.183515,0.189189,0.193694,0.200622,0.198198,0.202703,0.202703,222.0,643.0,865.0
b,0.121519,0.125000,0.136709,0.142640,0.153409,0.162025,0.163004,0.164557,0.164680,0.170455,0.173825,528.0,395.0,923.0
w,0.150605,0.156000,0.154724,0.152086,0.152086,0.149394,0.149394,0.150740,0.150000,0.152086,0.154000,743.0,500.0,1243.0
B,0.205607,0.198598,0.209396,0.214953,0.222412,0.226867,0.222992,0.221963,0.221557,0.224123,0.224979,428.0,1169.0,1597.0
D,0.181682,0.181522,0.189149,0.192391,0.201593,0.207609,0.213043,0.217279,0.219333,0.220010,0.221503,920.0,2009.0,2929.0
m,0.153989,0.145022,0.147186,0.147959,0.150146,0.151515,0.153150,0.152752,0.154519,0.154917,0.154519,3234.0,686.0,3920.0
i,0.254443,0.258481,0.257739,0.254413,0.255250,0.254616,0.255021,0.253635,0.254210,0.253446,0.252827,4929.0,1238.0,6167.0
s,0.325444,0.296492,0.299026,0.302846,0.302083,0.300993,0.300860,0.299694,0.298345,0.298743,0.300000,7555.0,480.0,8035.0


In [13]:
main(
    eers_dir + "features_combination_supervectors_dct_eers/",
    csvs_dir + "features_combination_supervectors_dct.csv",
    [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
)

,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,corrects,incorrects,total
G,0.162162,0.174184,0.175676,0.189189,0.190380,0.191291,0.192773,0.206843,0.207207,0.207207,0.211509,222.0,643.0,865.0
b,0.121519,0.126582,0.141282,0.151899,0.156962,0.162025,0.162025,0.164359,0.167089,0.172152,0.177215,528.0,395.0,923.0
w,0.150605,0.150442,0.151210,0.150000,0.156000,0.157738,0.158773,0.158000,0.160000,0.162853,0.163315,743.0,500.0,1243.0
B,0.205607,0.201882,0.203271,0.210280,0.213003,0.216424,0.217282,0.222295,0.224979,0.229256,0.231308,428.0,1169.0,1597.0
D,0.181682,0.182781,0.190144,0.201593,0.212359,0.218019,0.221982,0.223992,0.231458,0.230435,0.233449,920.0,2009.0,2929.0
m,0.153989,0.140723,0.145714,0.145462,0.144811,0.151020,0.152041,0.153370,0.153989,0.153989,0.153061,3234.0,686.0,3920.0
i,0.254443,0.251775,0.250404,0.253398,0.255224,0.260097,0.263131,0.263328,0.267367,0.268174,0.268174,4929.0,1238.0,6167.0
s,0.325444,0.283333,0.289583,0.294748,0.297816,0.298312,0.303111,0.304167,0.300000,0.299960,0.301787,7555.0,480.0,8035.0


# Step by step

In [19]:
base_dir = "/Users/lmatayoshi/Documents/Projects/tesis_notebooks/kappa_experiments/"
eers_dir = base_dir + "features_combination_supervectors_legendre_eers/"
csv_filename = base_dir + "features_combination_supervectors_legendre.csv"

In [22]:
proportions = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [23]:
eers_by_proportion = dict(zip(proportions, [None] * len(proportions)))

In [24]:
for p in proportions:
    with open(eers_dir + str(p), "r") as f:
        lines = f.readlines()
        eer_degree_i = [lines[j].strip().split(":") for j in range(len(lines))]
        eer_degree_i = [value for value in eer_degree_i if len(value) == 2]
        eer_degree_i = dict(eer_degree_i)
        eers_by_proportion[p] = eer_degree_i

In [25]:
kappa_total_dict, kappa_positives_dict, kappa_negatives_dict = kappa_positives_negatives_dict()
kappa_total = np.array([kappa_total_dict[phoneme] for phoneme in kappa_phonemes]).reshape(-1,1)
kappa_positives = np.array([kappa_positives_dict[phoneme] for phoneme in kappa_phonemes]).reshape(-1,1)
kappa_negatives = np.array([kappa_negatives_dict[phoneme] for phoneme in kappa_phonemes]).reshape(-1,1)

In [26]:
base_matrix = np.array(kappa_phonemes).reshape(-1,1)

In [27]:
for key in eers_by_proportion.keys():
    results_dict = eers_by_proportion[key]
    results = [float(results_dict[phoneme]) for phoneme in kappa_phonemes]
    results = np.array(results).reshape(-1,1)
    base_matrix = np.hstack((base_matrix, results))
    
base_matrix = np.hstack((base_matrix, kappa_positives))
base_matrix = np.hstack((base_matrix, kappa_negatives))
base_matrix = np.hstack((base_matrix, kappa_total))

In [28]:
dataframe = pd.DataFrame(base_matrix[:, 1:], index=kappa_phonemes, columns=proportions+['corrects', 'incorrects', 'total'])
dataframe = dataframe.astype(np.float)b
dataframe = dataframe.sort_values(by=['total'])
dataframe.round(3).to_csv(path_or_buf=csv_filename, index_label="Phonemes")


In [56]:
"_".join(['G', 'b', 'w', 'B', 'D', 'm', 'i', 's'])

'G_b_w_B_D_m_i_s'